<h2>Import NumPy and Check Python Version</h2>

First, you must import NumPy. Let's also check our version of Python. We've added the code for you for this first step.

In [1]:
import sys
import numpy as np
from pylab import * 
print('You\'re running python %s' % sys.version.split(' ')[0])

You're running python 3.9.12



<h2> Euclidean Distances in Python </h2>

<p>Many machine learning algorithms access their input data primarily through pairwise (Euclidean) distances, therefore it is important that we have a fast function that computes pairwise distances of input vectors.</p>
<p>Assume we have $n$ row data vectors $\mathbf{x_1}, \dots, \mathbf{x_n} \in \mathcal{R}^d$ and $m$ row vectors $\mathbf{z_1}, \dots, \mathbf{z_m} \in \mathcal{R}^d$. With these data vectors, let us define two matrices $X = \left[ \mathbf{x_1}^\top, \dots, \mathbf{x_n}^\top \right]^\top \in \mathcal{R}^{n \times d}$, where the $i^{th}$ row is vector $\mathbf{x_i}$, and $Z = \left[ \mathbf{z_1}^\top, \dots, \mathbf{z_m}^\top \right]^\top \in \mathcal{R}^{m \times d}$.We want a distance function that takes as input these two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where 
	$$D_{ij}=\sqrt{(\mathbf{x_i}-\mathbf{z_j})(\mathbf{x_i}-\mathbf{z_j})^\top}.$$
</p>

A naïve implementation to compute pairwise distances may look like the code below:

In [2]:
def l2distanceSlow(X,Z=None):
    if Z is None:
        Z = X
    
    n, d = X.shape     # dimension of X
    m= Z.shape[0]   # dimension of Z
    D=np.zeros((n,m)) # allocate memory for the output matrix
    for i in range(n):     # loop over vectors in X
        for j in range(m): # loop over vectors in Z
            D[i,j]=0.0; 
            for k in range(d): # loop over dimensions
                D[i,j]=D[i,j]+(X[i,k]-Z[j,k])**2; # compute l2-distance between the ith and jth vector
            D[i,j]=np.sqrt(D[i,j]); # take square root
    return D

This naive version is perfectly correct and will produce the correct result... eventually. To see what is wrong, run the <code>l2distanceSlow</code> code below on an extremely small matrix <code>X</code>.


In [3]:
X=np.random.rand(700,100)
print("Running the naive version, please wait...")
%time Dslow=l2distanceSlow(X)

Running the naive version, please wait...
CPU times: user 35.2 s, sys: 45.2 ms, total: 35.3 s
Wall time: 35.3 s


This code defines some random data in $X$ and computes the corresponding distance matrix $D$. The <em>%time</em> statement determines how long this code takes to run. This implementation is much too slow for such a simple operation on a small amount of data, and writing code like this to deal with matrices will result in code that takes <strong>days</strong> to run.





<h2> Efficient Programming with NumPy </h2>

<p>The key to efficient programming in Python for machine learning in general is to think about it in terms of mathematics and not in terms of loops. </p>

<h3>Inner-Product Matrix</h3>

<p>The Inner-Product Matrix (Gram matrix) can be expressed in terms of pure matrix multiplication.

$$	G_{ij}=\mathbf{x}_i\mathbf{z}_j^\top $$

In [4]:
def innerproduct(X,Z=None):
    '''
    function innerproduct(X,Z)
    
    Computes the inner-product matrix.
    Syntax:
    D=innerproduct(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix G of size nxm
    G[i,j] is the inner-product between vectors X[i,:] and Z[j,:]
    
    call with only one input:
    innerproduct(X)=innerproduct(X,X)
    '''
    if Z is None: # case when there is only one input (X)
        Z=X;
    
    # G Matrix
    
    # X is an nxd matrix
    # Z is an mxd matrix, so Z.T is a dxm matrix
    # Now with matching last/first dimensions, I can take the dot product of these matrices
    
    # The (ij) element in the Gram matrix needs to be the dot product of the (i) component and the transpose of the
    # (j) component of Z matrix.
    # I pictured X as a single column of length n, where each component is an array of length d.
    # I pictured Z as a single column of length m, where each component is an array of length d as well.
    # By taking the transpose of Z, I transform it into a row, and the dot product of X and Z.T has dimensions nxm.
    
    G = X @ Z.T
    return G

In [5]:
# Simple test case
X = np.array([[1,2],[3,4]])
Z = np.array([[1,4],[2,5],[3,6]])

print(innerproduct(X,Z))

[[ 9 12 15]
 [19 26 33]]


### Implement `calculate_S` and `calculate_R`

Let us define two new matrices, $S, R \in \mathcal{R}^{n \times m}$
$$
S_ij = \mathbf{x}_i \mathbf{x}_i^\top, \ \ R_{ij} = \mathbf{z}_j \mathbf{z}_j^\top
$$

We have shown that the _squared_-euclidean matrix $D^2 \in \mathcal{R}^{n \times m}$, defined as
$$
D_{ij}^2 = (\mathbf{x}_i - \mathbf{z}_j)(\mathbf{x}_i - \mathbf{z}_j)^\top,
$$
can be expressed as
$$
D^2 = S + R - 2G.
$$

The `l2distance` will calculate $D$, which needs $S$ and $R$, which are found in **`calculate_S`** and **`calculate_R`** respectively. These functions return $S$ and $R$ of size $n \times m$, so they can be added to $-2G$ to get $D^2$.

The values in each row of $S$ and the values in each column of $R$ do not change! This is also apparent when considering that $S_{ij} = \mathbf{x}_i \mathbf{x}_i^\top$ _for all $j$_; similar argument for $R_{ij} = \mathbf{z}_j \mathbf{z}_j^\top$ _for all $i$_.
$$
S = \begin{bmatrix}
\mathbf{x}_1 \mathbf{x}_1^\top & \mathbf{x}_1 \mathbf{x}_1^\top & \cdots & \mathbf{x}_1 \mathbf{x}_1^\top\\
\mathbf{x}_2 \mathbf{x}_2^\top & \mathbf{x}_2 \mathbf{x}_2^\top & \cdots & \mathbf{x}_2 \mathbf{x}_2^\top\\
\vdots & \vdots & \ddots & \vdots\\
\mathbf{x}_n \mathbf{x}_n^\top & \mathbf{x}_n \mathbf{x}_n^\top & \cdots & \mathbf{x}_n \mathbf{x}_n^\top\\
\end{bmatrix}, \ 
R = \begin{bmatrix}
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\vdots & \vdots & \ddots & \vdots\\
\mathbf{z}_1 \mathbf{z}_1^\top & \mathbf{z}_2 \mathbf{z}_2^\top & \cdots & \mathbf{z}_m \mathbf{z}_m^\top\\
\end{bmatrix}.
$$

In [6]:
def calculate_S(X, n, m):
    '''
    function calculate_S(X)
    
    Computes the S matrix.
    Syntax:
    S=calculate_S(X)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    n: number of rows in X
    m: output number of columns in S
    
    Output:
    Matrix S of size nxm
    S[i,j] is the inner-product between vectors X[i,:] and X[i,:]
    '''
    assert n == X.shape[0]
    
    # S Matrix
    # S has identical columns all the way across (need to find one column, n x 1)
    # (each of the n rows is filled with m repeats of the same value, (xi)(xi).T
    
    # To find this column which will be repeated across the S matrix, I can take the dot product of X and X-transpose.
    # The diagonal of this nxn matrix contains the elements I need, since the indices need to match: (xi)(xi).T
    col = (X @ X.T).diagonal()
    #print(col)
    
    # This mechanism (below) creates a matrix with a repeated column
    S = np.array([col]*m).T 
    return S
    

In [7]:
# Simple test case
# X = [[1,2],[3,4]]
# Z = [[1,4],[2,5],[3,6]]

calculate_S(X,2,3)

array([[ 5,  5,  5],
       [25, 25, 25]])

In [8]:
def calculate_R(Z, n, m):
    '''
    function calculate_R(Z)
    
    Computes the R matrix.
    Syntax:
    R=calculate_R(Z)
    Input:
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    n: output number of rows in Z
    m: number of rows in Z
    
    Output:
    Matrix R of size nxm
    R[i,j] is the inner-product between vectors Z[j,:] and Z[j,:]
    '''
    assert m == Z.shape[0]
    
    # R Matrix
    # R has identical rows all the way down (need to find one row, 1 x m)
    # (each of the m columns is filled with n repeats of the same value, (zj)(zj).T
    
    # The same logic applies here as in the S matrix calculation. I want to find a single row of values which will
    # be repeated for the whole R matrix.
    row = (Z @ Z.T).diagonal()
    
    R = np.array([row]*n)
    return R

In [9]:
# Simple test case
# X = [[1,2],[3,4]]
# Z = [[1,4],[2,5],[3,6]]
calculate_R(Z,2,3)

array([[17, 29, 45],
       [17, 29, 45]])

<h3>Implement <code>l2distance</code></h3>

$$
D^2_{ij}=(\mathbf{x}_i-\mathbf{z}_j)(\mathbf{x}_i-\mathbf{z}_j)^\top, \ D^2 = S + R - 2G
$$
    
<p> The above formula calculates the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop. </p>

<p><strong>Note</strong>: Sometimes very small positive numbers can become negative due to <em>numerical imprecision</em>. Knowing that all distances must always be non-negative, all negative values are overwritten as 0.0 to avoid unintended consequences. </p>

In [10]:
def l2distance(X,Z=None):
    '''
    function D=l2distance(X,Z)
    
    Computes the Euclidean distance matrix.
    Syntax:
    D=l2distance(X,Z)
    Input:
    X: nxd data matrix with n vectors (rows) of dimensionality d
    Z: mxd data matrix with m vectors (rows) of dimensionality d
    
    Output:
    Matrix D of size nxm
    D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    
    call with only one input:
    l2distance(X)=l2distance(X,X)
    '''
    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    assert (d1==d2), "Dimensions of input vectors must match!"
    
    # D^2 = S + R - 2G
    
    S = calculate_S(X,n,m)
    R = calculate_R(Z,n,m)
    G = innerproduct(X,Z)
    
    D_squared = S + R - 2*G
    
    # D Matrix
    # need to get rid of negative values in D^2 and take the square root
    
    # (1) D_squared.shape : finds the dimensions of the D_squared matrix (nxm)
    # (2) np.zeros(1) : creates an nxm matrix of all 0s
    # (3) np.maximum(D_squared,2) : Compares each value in the D_squared matrix to 0, and chooses the higher of the 
    # two values. If there are negative values in D_squared, 0 will be greater than that value and will be chosen.
    # (4) np.sqrt(3) : take square root of all elements in D_squared matrix
    
    D = np.sqrt(np.maximum(D_squared,np.zeros(D_squared.shape)))
    return D

In [11]:
# Simple test case
# X = [[1,2],[3,4]]
# Z = [[1,4],[2,5],[3,6]]
l2distance(X,Z)

array([[2.        , 3.16227766, 4.47213595],
       [2.        , 1.41421356, 2.        ]])

Let's now compare the speed of your l2-distance function against the previous naïve implementation:

In [12]:
import time
current_time = lambda: int(round(time.time() * 1000))

X=np.random.rand(700,100)
Z=np.random.rand(300,100)

print("Running the naïve version...")
before = current_time()
Dslow=l2distanceSlow(X)
after = current_time()
t_slow = after - before
print("{:2.0f} ms".format(t_slow))

print("Running the vectorized version...")
before = current_time()
Dfast=l2distance(X)
after = current_time()
t_fast = after - before
print("{:2.0f} ms".format(t_fast))


speedup = t_slow / t_fast
print("The two methods should deviate by very little: {:05.6f}".format(norm(Dfast-Dslow)))
print("but your NumPy code was {:05.2f} times faster!".format(speedup))

Running the naïve version...
34362 ms
Running the vectorized version...
21 ms
The two methods should deviate by very little: 0.000000
but your NumPy code was 1636.29 times faster!


How much faster is your code now? With this implementation, you should easily be able to compute the distances between <strong>many more</strong> vectors. It should be clear now, even for small datasets, that the for-loop based implementation could take several days or even weeks to perform basic operations that take seconds or minutes with well-written NumPy code.